# Implement RAG with Knowledge Graph and Llama-Index

Hallucination is a common problem when working with large language models (LLMs). LLMs generate fluent and coherent <br>
text but often generate inaccurate or inconsistent information. One of the ways to prevent hallucination in LLMs is <br> 
by using external knowledge sources such as databases or knowledge graphs that provide factual information.

In [1]:
import yaml, logging, sys, os
from pyvis.network import Network
from IPython.display import display
from llama_index.llms import AzureOpenAI
from llama_index.llm_predictor import LLMPredictor
from llama_index import set_global_service_context
from llama_index.graph_stores import SimpleGraphStore
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.storage.storage_context import StorageContext
from llama_index import (
                        SimpleDirectoryReader, 
                        load_graph_from_storage,
                        KnowledgeGraphIndex,
                        ServiceContext
                        )

logging.basicConfig(
                    stream=sys.stdout, 
                    level=logging.INFO
                    )
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [2]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

# Configuring LLMs

In [3]:
embedding_llm = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm=AzureOpenAI(
                model=credentials['AZURE_ENGINE'],
                api_key=credentials['AZURE_OPENAI_API_KEY'],
                deployment_name=credentials['AZURE_DEPLOYMENT_ID'],
                api_version=credentials['AZURE_OPENAI_API_VERSION'],
                azure_endpoint=credentials['AZURE_OPENAI_API_BASE']
                )
chat_llm = LLMPredictor(llm)

graph_store = SimpleGraphStore()

service_context = ServiceContext.from_defaults(
                                                chunk_size=1000,
                                                embed_model=embedding_llm,
                                                llm_predictor=chat_llm
                                                )
storage_context = StorageContext.from_defaults(graph_store=graph_store)

set_global_service_context(service_context)

/Users/1zuu/miniforge3/envs/llamaindex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LLMPredictor is deprecated, please use LLM instead.


# Load Data and Vector Storing

In [4]:
# Load Data
documents = SimpleDirectoryReader("./data/new_articles").load_data()
print(len(documents))

21


- `max_triplets_per_chunk` : it governs the number of relationship triplets processed per data chunk
- `include_embeddings` : toggles the inclusion of vector embeddings within the index for advanced analytics.

In [5]:
if not os.path.exists('./db/03/'):
    kg_index = KnowledgeGraphIndex.from_documents( 
                                            documents=documents,
                                            max_triplets_per_chunk=12,
                                            service_context=service_context,
                                            storage_context=storage_context,
                                            include_embeddings=True
                                            )
    
    kg_index.storage_context.persist(persist_dir='./db/03/')

else:
    kg_index = load_graph_from_storage(storage_context=storage_context.persist(persist_dir='./db/03/'))

INFO:httpx:HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model

# Querying

In [6]:
query_engine = kg_index.as_query_engine(
                                        include_text=True,
                                        response_mode ="tree_summarize",
                                        embedding_mode="hybrid",
                                        similarity_top_k=5
                                        )

In [10]:
query = "What is bing AI ?"

message_template =f"""<|system|>Please check if the following pieces of context has any mention of the  keywords provided in the Question.If not then don't know the answer, just say that you don't know.Stop there.Please donot try to make up an answer.</s>
<|user|>
Question: {query}
Helpful Answer:
</s>"""

response = query_engine.query(query)
response

INFO:httpx:HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 058565bf-18c7-4ea2-9b39-826837364a15: Microsoft doubles down on AI with new Bing features The company's betting the...
> Querying with idx: 058565bf-18c7-4ea2-9b39-826837364a15: Microsoft doubles down on AI with new Bing features The company's betting the...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: 7f13f2e3-92e4-4427-9388-984f2b70a1d8: Any image upload tool can be abused, of course, which is why Microsoft is emp...
> Querying with idx: 7f13f2e3-92e4-4427-9388-984f2b70a1d8: Any image upload tool can be abused, of course, which is why Microsoft is emp...
INFO:llama_index.indices.knowledge_

Response(response="The context information mentions that Bing AI is a chatbot launched by Microsoft that is now available to all users globally. It heavily revolves around AI and Microsoft has integrated OpenAI's GPT-4 model into Bing. Bing Chat, which is powered by OpenAI's GPT-4 and DALL-E 2 models, allows for multimodality, meaning it can understand queries with images and text combined, and can create charts and graphs when fed the right prompt and data. Additionally, Bing Chat is getting plugins from partners like OpenTable and Wolfram Alpha, and Actions can translate Bing Chat prompts into automations. Bing AI also allows users to export their Bing Chat histories and can generate images from text prompts.", source_nodes=[NodeWithScore(node=TextNode(id_='058565bf-18c7-4ea2-9b39-826837364a15', embedding=None, metadata={'file_path': 'data/new_articles/05-04-microsoft-doubles-down-on-ai-with-new-bing-features.txt', 'file_name': '05-04-microsoft-doubles-down-on-ai-with-new-bing-featur

# Graph Visualizations

In [8]:
g = kg_index.get_networkx_graph()
net = Network(
            notebook=True,
            cdn_resources="in_line",
            directed=True
            )
net.from_nx(g)
net.show("./vis/03/graph.html")
net.save_graph("./vis/03/Knowledge_graph.html")

./vis/03/graph.html
